In [1]:
import json
import geopandas as gpd
import pandas as pd
import numpy as np

# Cultural Equipments Dataset Wrangling

In [2]:
pd.set_option("display.max_columns", 999)
ce_gdf = gpd.read_file("./Data/equipCulturels.geojson")
ce_gdf.head()

,orgaweb,domanetlibelle1,domanetlibelle2,domanetlibelle3,coortel1,orgahoraires,adrenumero,communeid,orgainfo,quarnom,coorfax,coorcp,commnom,latitude,orgasigle,orgadesc,accessibilite_1,quarnum,orgadomanetdescription,rivolide,longitude,coorbp,adreextension,organismeid,organom,adrerue,orgaemail,accessibilite_2,accessibilite_4x,accessibilite_3,coortel2,domanetdescription3,domanetdescription2,domanetlibelle4,adreimmeuble,geometry
0,www.cerclepaulbert.asso.fr,LOISIRS CULTURELS,"Enfants, adolescents",Les accueils de loisirs associatifs,02 99 27 02 46,"Lundi, mardi et jeudi, 14h à 18h. Mercredi, 9h...",30.0,1,Accompagnement scolaire individualisé en colle...,Thabor - Saint-Hélier - Alphonse Guérin,02 99 27 74 09,35704,Rennes,48.11199639,CPB,Equipement de quartier : activités sportives e...,Non renseigné,2.0,Centre de loisirs associatifs pour les 3-12ans...,RUE DE PARIS,-1.667547278,BP 60401,bis,1036,Cercle Paul Bert Centre,RUE DE PARIS,centre@cerclepaulbert.asso.fr,None,None,None,None,None,None,None,None,POINT (-1.66755 48.11200)
1,www.cerclepaulbert.asso.fr,LOISIRS CULTURELS,"Enfants, adolescents",Les accueils de loisirs associatifs,02 99 63 19 13,"Lundi, mardi, jeudi et vendredi : 10h à 12h30 ...",12.0,1,Accompagnement à la scolarité collectif de la ...,Maurepas - Bellangerais,02 99 36 54 06,35700,Rennes,48.13198222,CPB,Association affiliée à la Ligue Française de l...,handicap auditif : bonne accessibilité,5.0,Accueil de loisirs pour ados.,RUE DES LONGS PRES,-1.654075436,None,None,646,Cercle Paul Bert Gayeulles,RUE DES LONGS PRES,gayeulles@cerclepaulbert.asso.fr,handicap cognitif : bonne accessibilité,handicap visuel : accessibilité moyenne,handicap moteur : bonne accessibilité,None,None,None,None,None,POINT (-1.65408 48.13198)
2,www.educarennes.fr/index.php/public/default/ec...,LOISIRS CULTURELS,"Enfants, adolescents","Mercredis et petites vacances, les accueils de...",None,De 12h15 à 18h45,5.0,1,None,Sud-Gare,None,35000,Rennes,48.1017143864,None,"Accueil de loisirs le mercredi après-midi, de ...",None,8.0,None,None,-1.6668462590,None,None,3012,Accueil de loisirs Albert de Mun /Quineleu,Rue Albert Martin,None,None,None,None,None,None,None,None,None,POINT (-1.66685 48.10171)
3,www.bibliotheques.rennes.fr,LOISIRS CULTURELS,Tout public,Bibliothèques municipales,02 23 62 26 69,"Mardi et jeudi, 16h30 à 18h30. Mercredi, 10h à...",123.0,1,Contact Maison bleue : 02 23 33 75 25 ou Maiso...,Saint-Martin,None,35000,Rennes,48.122952,None,Emprunts de livres et animations autour du liv...,Non renseigné,4.0,None,BOULEVARD DE VERDUN,-1.685703696,None,None,1392,Espace lecture Saint-Martin,None,bibliotheque.smae@ville-rennes.fr,None,None,None,02 23 33 75 25,11 bibliothèques ouvertes à tous vous accueill...,La plupart des équipements de quartier (munici...,None,None,POINT (-1.68570 48.12295)
4,www.bibliotheques.rennes.fr,LOISIRS CULTURELS,Tout public,Bibliothèques municipales,02 23 62 26 36,"Mardi et vendredi, 14h30 à 18h30. Mercredi, 10...",60.0,1,None,Jeanne d'Arc - Longs Champs - Beaulieu,None,35031,Rennes,48.12804386,None,"A Rennes, 11 bibliothèques de quartier ouverte...",Non renseigné,6.0,None,RUE DOYEN ALBERT ET PIERRE BOUZAT,-1.635549791,CS 63126,None,356,Bibliothèque Longs Champs,None,bibliotheque.longs-champs@ville-rennes.fr,None,None,None,None,11 bibliothèques ouvertes à tous vous accueill...,La plupart des équipements de quartier (munici...,None,None,POINT (-1.63555 48.12804)


We got a lot of cleaning to do ...

In [3]:
# We can drop the phone and fax coordinates
ce_gdf.drop(['coortel1', 'coortel2', 'coorfax'], axis=1, inplace=True)
# As well as the few columns that gives the adress
ce_gdf.drop(['adrenumero', 'communeid', 'coorcp', 'commnom', 'orgasigle', 'rivolide', 'coorbp', 'adreextension', 'organismeid', 'adrerue', 'adreimmeuble'], axis=1, inplace=True)

In [4]:
# for simplicity sake, let's get rid of data regarding accessibility
# even if it could be useful in the future
ce_gdf.drop(['accessibilite_1', 'accessibilite_2', 'accessibilite_2', 'accessibilite_4x', 'accessibilite_3'], axis=1, inplace=True)
ce_gdf.shape

(248, 18)

In [5]:
# Let's drop the Neighborhood column as we already have the Borough Number
ce_gdf.drop(['quarnom'], axis=1, inplace=True)
ce_gdf.dropna(axis=0, subset=['quarnum'], inplace=True)
ce_gdf['quarnum'] = ce_gdf['quarnum'].astype('int')

The cultural equipment description, and information regarding the organisation in charge of them,
is described in columns :
    - domanetlibelle3
    - orgainfo
    - orgadesc
    - orgadomanetdescription
    - domanetdescription
    - domanetdescription2
    - domanetdescription3
    - domanetlibelle4
    
Only the first one is useful, the others are either too long, redundant, or filled with a lot of None values

In [6]:
ce_gdf.drop(['orgainfo', 'orgadesc', 'orgadomanetdescription', 'domanetdescription2', 'domanetdescription3', 'domanetlibelle4'], axis=1, inplace=True)

In [7]:
ce_gdf.domanetlibelle3.unique()

array(['Les accueils de loisirs associatifs',
       'Mercredis et petites vacances, les accueils de loisirs municipaux',
       'Bibliothèques municipales',
       'Bibliothèques spécialisées et autres',
       'Les équipements de quartier qui proposent des activités régulières',
       'Les archives', "Les accueils de loisirs d'été",
       'Les pôles multimédia',
       'Les structures de développement artistique',
       'Où trouver les informations ?',
       "Centres d'animation nature-environnement (Ville de Rennes)",
       "Les centres d'accueil pour enfants handicapés"], dtype=object)

In [8]:
# Let's translate the data
ce_gdf.loc[(ce_gdf['domanetlibelle3'] == 'Les accueils de loisirs associatifs'), 'domanetlibelle3'] = 'Community leisure centers'
ce_gdf.loc[(ce_gdf['domanetlibelle3'] == 'Mercredis et petites vacances, les accueils de loisirs municipaux'), 'domanetlibelle3'] = 'Wednesday and small-vacations municipal recreation centers'
ce_gdf.loc[(ce_gdf['domanetlibelle3'] == 'Bibliothèques municipales'), 'domanetlibelle3'] = 'Municipal libraries'
ce_gdf.loc[(ce_gdf['domanetlibelle3'] == 'Bibliothèques spécialisées et autres'), 'domanetlibelle3'] = 'Specialized libraries and others'
ce_gdf.loc[(ce_gdf['domanetlibelle3'] == 'Les équipements de quartier qui proposent des activités régulières'), 'domanetlibelle3'] = 'Neighborhood equipments offering regular activities'
ce_gdf.loc[(ce_gdf['domanetlibelle3'] == 'Les archives'), 'domanetlibelle3'] = 'Archives'
ce_gdf.loc[(ce_gdf['domanetlibelle3'] == "Les accueils de loisirs d'été"), 'domanetlibelle3'] = 'Summer laisure centers'
ce_gdf.loc[(ce_gdf['domanetlibelle3'] == 'Les pôles multimédia'), 'domanetlibelle3'] = 'Multimedia departments'
ce_gdf.loc[(ce_gdf['domanetlibelle3'] == 'Les structures de développement artistique'), 'domanetlibelle3'] = 'Artistic development structures'
ce_gdf.loc[(ce_gdf['domanetlibelle3'] == 'Où trouver les informations ?'), 'domanetlibelle3'] = 'Information point'
ce_gdf.loc[(ce_gdf['domanetlibelle3'] == "Centres d'animation nature-environnement (Ville de Rennes)"), 'domanetlibelle3'] = 'Nature-Environment animation center (City of Rennes)'
ce_gdf.loc[(ce_gdf['domanetlibelle3'] == "Les centres d'accueil pour enfants handicapés"), 'domanetlibelle3'] = 'Activity center for handicapped children'

In [9]:
# same work for the 'domanetlibelle2' column with targeted audience
ce_gdf.loc[(ce_gdf['domanetlibelle2'] == 'Enfants, adolescents'), 'domanetlibelle2'] = 'Childs, Teenagers'
ce_gdf.loc[(ce_gdf['domanetlibelle2'] == 'Tout  public'), 'domanetlibelle2'] = 'All Audiences' # 2 spaces ... good job guys
ce_gdf.loc[(ce_gdf['domanetlibelle2'] == 'Pratiquer une activité culturelle'), 'domanetlibelle2'] = 'Practice a cultural activity'
ce_gdf.rename(columns={'domanetlibelle2':'Audiences'}, inplace=True)

In [10]:
ce_gdf.head()

,orgaweb,domanetlibelle1,Audiences,domanetlibelle3,orgahoraires,latitude,quarnum,longitude,organom,orgaemail,geometry
0,www.cerclepaulbert.asso.fr,LOISIRS CULTURELS,"Childs, Teenagers",Community leisure centers,"Lundi, mardi et jeudi, 14h à 18h. Mercredi, 9h...",48.11199639,2,-1.667547278,Cercle Paul Bert Centre,centre@cerclepaulbert.asso.fr,POINT (-1.66755 48.11200)
1,www.cerclepaulbert.asso.fr,LOISIRS CULTURELS,"Childs, Teenagers",Community leisure centers,"Lundi, mardi, jeudi et vendredi : 10h à 12h30 ...",48.13198222,5,-1.654075436,Cercle Paul Bert Gayeulles,gayeulles@cerclepaulbert.asso.fr,POINT (-1.65408 48.13198)
2,www.educarennes.fr/index.php/public/default/ec...,LOISIRS CULTURELS,"Childs, Teenagers",Wednesday and small-vacations municipal recrea...,De 12h15 à 18h45,48.1017143864,8,-1.6668462590,Accueil de loisirs Albert de Mun /Quineleu,None,POINT (-1.66685 48.10171)
3,www.bibliotheques.rennes.fr,LOISIRS CULTURELS,All Audiences,Municipal libraries,"Mardi et jeudi, 16h30 à 18h30. Mercredi, 10h à...",48.122952,4,-1.685703696,Espace lecture Saint-Martin,bibliotheque.smae@ville-rennes.fr,POINT (-1.68570 48.12295)
4,www.bibliotheques.rennes.fr,LOISIRS CULTURELS,All Audiences,Municipal libraries,"Mardi et vendredi, 14h30 à 18h30. Mercredi, 10...",48.12804386,6,-1.635549791,Bibliothèque Longs Champs,bibliotheque.longs-champs@ville-rennes.fr,POINT (-1.63555 48.12804)


In fact let's get rid of all the information about the organization in charge of the equipments. We only need to know what and for who.

In [11]:
ce_gdf.drop(['orgaweb', 'organom', 'orgaemail'], axis=1, inplace=True)

In [12]:
# The 'domanetlibelle1' can be drop as it only contains one value
ce_gdf.drop(['domanetlibelle1'], axis=1, inplace=True)

In [13]:
# Let's rename the remaining columns
ce_gdf.rename(columns={'domanetlibelle3':'Venue Category', 'orgahoraires':'Hours', 'quarnum':'Borough_ID', 'longitude':'Longitude', 'latitude':'Latitude'}, inplace=True)

In [14]:
ce_gdf.head()

,Audiences,Venue Category,Hours,Latitude,Borough_ID,Longitude,geometry
0,"Childs, Teenagers",Community leisure centers,"Lundi, mardi et jeudi, 14h à 18h. Mercredi, 9h...",48.11199639,2,-1.667547278,POINT (-1.66755 48.11200)
1,"Childs, Teenagers",Community leisure centers,"Lundi, mardi, jeudi et vendredi : 10h à 12h30 ...",48.13198222,5,-1.654075436,POINT (-1.65408 48.13198)
2,"Childs, Teenagers",Wednesday and small-vacations municipal recrea...,De 12h15 à 18h45,48.1017143864,8,-1.6668462590,POINT (-1.66685 48.10171)
3,All Audiences,Municipal libraries,"Mardi et jeudi, 16h30 à 18h30. Mercredi, 10h à...",48.122952,4,-1.685703696,POINT (-1.68570 48.12295)
4,All Audiences,Municipal libraries,"Mardi et vendredi, 14h30 à 18h30. Mercredi, 10...",48.12804386,6,-1.635549791,POINT (-1.63555 48.12804)


In [15]:
# And finally, let's reorganize the columns
fixed_columns = ['Venue Category', 'Borough_ID', 'Audiences', 'Hours', 'Latitude', 'Longitude', 'geometry']

ce_gdf = ce_gdf[fixed_columns]
ce_gdf.head()

,Venue Category,Borough_ID,Audiences,Hours,Latitude,Longitude,geometry
0,Community leisure centers,2,"Childs, Teenagers","Lundi, mardi et jeudi, 14h à 18h. Mercredi, 9h...",48.11199639,-1.667547278,POINT (-1.66755 48.11200)
1,Community leisure centers,5,"Childs, Teenagers","Lundi, mardi, jeudi et vendredi : 10h à 12h30 ...",48.13198222,-1.654075436,POINT (-1.65408 48.13198)
2,Wednesday and small-vacations municipal recrea...,8,"Childs, Teenagers",De 12h15 à 18h45,48.1017143864,-1.6668462590,POINT (-1.66685 48.10171)
3,Municipal libraries,4,All Audiences,"Mardi et jeudi, 16h30 à 18h30. Mercredi, 10h à...",48.122952,-1.685703696,POINT (-1.68570 48.12295)
4,Municipal libraries,6,All Audiences,"Mardi et vendredi, 14h30 à 18h30. Mercredi, 10...",48.12804386,-1.635549791,POINT (-1.63555 48.12804)


In [16]:
ce_gdf.to_file("./Data/cleaned_data/cultural_equipments.geojson", driver='GeoJSON')